In [1]:
import copy

def e(inval, part):
    inst = [x.strip().split(" ") for x in open("23.txt").readlines()]
    
    orig = copy.deepcopy(inst)
    
    def pinst(i):
        r = ""
        for ii in range(3):
            if ii < len(i):
                r+= "% 3s"%(i[ii])
            else:
                r+= "   "
            r+= " "
        return r
    
    def pi(orig, inst, cnt, tc):
        for x in range(len(inst)):
            r = "% 3d  (% 5d) (% 5d) "%(x, cnt[x], tc[x])
            r+=pinst(orig[x])
            r += " | "
            r+=pinst(inst[x])
            if inst[x] != orig[x]:
                r+= " <----"
            print(r)
        print()
    
    #
    #
    #
    
    
    #remove inner loop
    if 0:
        inst[5] = ["add", 'c', 'a']
        inst[6] = ["nop"]
        inst[7] = ["cpy", '0', 'c']
    
    
#  0  (    1) (    0) cpy   a   b  | cpy   a   b 
#  1  (    1) (    0) dec   b      | dec   b     
#  2  (    5) (    0) cpy   a   d  | cpy   a   d 
#  3  (    5) (    0) cpy   0   a  | cpy   0   a 
#  4  ( 3619) (    0) cpy   b   c  | cpy   b   c 
#  5  ( 8652) (    0) inc   a      | inc   a     
#  6  ( 8652) (    0) dec   c      | dec   c     
#  7  ( 8652) (    0) jnz   c  -2  | jnz   c  -2 
#  8  ( 3619) (    0) dec   d      | dec   d     
#  9  ( 3619) (    0) jnz   d  -5  | jnz   d  -5 
# 10  (    5) (    0) dec   b      | dec   b     
# 11  (    5) (    0) cpy   b   c  | cpy   b   c 
# 12  (    5) (    0) cpy   c   d  | cpy   c   d 
# 13  (   15) (    0) dec   d      | dec   d     
# 14  (   15) (    0) inc   c      | inc   c     
# 15  (   15) (    0) jnz   d  -2  | jnz   d  -2 
# 16  (    5) (    0) tgl   c      | tgl   c     
# 17  (    5) (    0) cpy -16   c  | cpy -16   c 
# 18  (    5) (    1) jnz   1   c  | cpy   1   c  <----
# 19  (    1) (    0) cpy  79   c  | cpy  79   c 
# 20  (   79) (    1) jnz  74   d  | cpy  74   d  <----
# 21  ( 5846) (    0) inc   a      | inc   a     
# 22  ( 5846) (    1) inc   d      | dec   d      <----
# 23  ( 5846) (    0) jnz   d  -2  | jnz   d  -2 
# 24  (   79) (    1) inc   c      | dec   c      <----
# 25  (   79) (    0) jnz   c  -5  | jnz   c  -5         


#remove entire loop
    if 1:
        inst[5] = ["mul", 'd', 'c']
        inst[6] = ["cpy", 'c', 'a']
        inst[7] = ["cpy", '0', 'c']
        inst[8] = ["cpy", '0', 'd']
        inst[9] = ["nop"]
    
    
    #print(inst)
    
    instctr = [0 for x in range(len(inst))]
    tctr = [0 for x in range(len(inst))]
    
    i=0
    reg={'a':inval,'b':0,'c':0,'d':0}
    
    #get value from register if possible else parse as number
    def gt(s):
        if s in reg:
            return reg[s]
        return int(s)
    
    #Continue while instruction pointer (i) is within code
    while i < len(inst):
        ix=inst[i]
        instctr[i]+=1
        
        if ix[0]=="cpy":
            a,b=ix[1:]
            #we can only write to registers
            if b in reg.keys():
                reg[b]=gt(a)
            i+=1
        
        elif ix[0]=="inc":
            reg[ix[1]]+=1
            i+=1
        
        elif ix[0]=="dec":
            reg[ix[1]]-=1
            i+=1
        
        elif ix[0]=="jnz":
            a,b=ix[1:]
            a=gt(a)
            b=gt(b)
            if a != 0:
                i+=b
            else:
                i+=1
                
        elif ix[0]=="tgl":
            #find target instruction to toggle
            tix = i+gt(ix[1])
            #print("tgl", tix)
            #only adresses in program
            if tix >= 0 and tix < len(inst):
                tctr[tix] += 1
                if len(inst[tix]) == 2:
                    if inst[tix][0] == "inc":
                        inst[tix][0]="dec"
                    else:
                        inst[tix][0]="inc"
                else:
                    if inst[tix][0] == "jnz":
                        inst[tix][0]="cpy"
                    else:
                        inst[tix][0]="jnz"
            i+=1
        elif ix[0]=="add":
            a,b=ix[1:]
            a=gt(a)
            if b in reg:
                reg[b]+=a
            i+=1
        elif ix[0]=="mul":
            a,b=ix[1:]
            a=gt(a)
            if b in reg:
                reg[b]*=a
            i+=1
        elif ix[0]=="nop":
            i+=1
        else:
            print("unknown instruction", ix)
    
    pi(orig, inst, instctr, tctr)
    
    
    return reg['a']



#print("Test:", e(6, 1))
print("Part1:", e(7, 1), 10886)
print("Part2:", e(12, 2)) #5978
print("fin")



  0  (    1) (    0) cpy   a   b  | cpy   a   b 
  1  (    1) (    0) dec   b      | dec   b     
  2  (    5) (    0) cpy   a   d  | cpy   a   d 
  3  (    5) (    0) cpy   0   a  | cpy   0   a 
  4  (    5) (    0) cpy   b   c  | cpy   b   c 
  5  (    5) (    0) inc   a      | mul   d   c  <----
  6  (    5) (    0) dec   c      | cpy   c   a  <----
  7  (    5) (    0) jnz   c  -2  | cpy   0   c  <----
  8  (    5) (    0) dec   d      | cpy   0   d  <----
  9  (    5) (    0) jnz   d  -5  | nop          <----
 10  (    5) (    0) dec   b      | dec   b     
 11  (    5) (    0) cpy   b   c  | cpy   b   c 
 12  (    5) (    0) cpy   c   d  | cpy   c   d 
 13  (   15) (    0) dec   d      | dec   d     
 14  (   15) (    0) inc   c      | inc   c     
 15  (   15) (    0) jnz   d  -2  | jnz   d  -2 
 16  (    5) (    0) tgl   c      | tgl   c     
 17  (    5) (    0) cpy -16   c  | cpy -16   c 
 18  (    5) (    1) jnz   1   c  | cpy   1   c  <----
 19  (    1) (    0) cpy  79   c 

In [2]:
#code does alot of unnecsesary loops


#  0   cpy a b              b=a-1
#  1   dec b                
#  2   cpy a d              d=a
#  3   cpy 0 a                
#  4   cpy b c  <--------     
#  5   inc a    <----   l   add c a  
#  6   dec c        l   l   nop
#  7   jnz c -2  ----   l   cpy 0 c  
#  8   dec d            l   
#  9   jnz d -5  --------     
# 10   dec b
# 11   cpy b c                
# 12   cpy c d                
# 13   dec d     <-------
# 14   inc c            l                  
# 15   jnz d -2  --------       
# 16   tgl c
# 17   cpy -16 c
# 18   jnz 1 c   --------------------->
# 19   cpy 79 c
# 20   jnz 74 d  --->OUT    <-----
# 21   inc a     <--------       l      
# 22   inc d             l       l
# 23   jnz d -2  ---------       l   
# 24   inc c                     l
# 25   jnz c -5  -----------------




